<a href="https://colab.research.google.com/github/ductt-1167/QuantizationTutorial/blob/main/Tensorflow_Scikitlearn_To_ONNX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tf2onnx
!pip install onnxruntime

# Converting Tensorflow to ONNX with basic CNN and MNIST dataset 

**Build model**

In [ ]:
import numpy as np
import tensorflow as tf

In [3]:
MODEL_TF_DIR = "/content/drive/MyDrive/Colab Notebooks/ONNX/model_MNIST"
MODEL_TF2ONNX_DIR = "/content/model_MNIST_tf.onnx"

In [4]:
#@title Build model
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(28, 28), name='input'),
    tf.keras.layers.LSTM(20, time_major=False, return_sequences=True),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax', name='output')
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 28, 20)            3920      
                                                                 
 flatten (Flatten)           (None, 560)               0         
                                                                 
 output (Dense)              (None, 10)                5610      
                                                                 
Total params: 9,530
Trainable params: 9,530
Non-trainable params: 0
_________________________________________________________________


In [5]:
#@title load MNIST dataset and training model
# Load MNIST dataset.
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)

print(x_train.shape, type(x_train))

# fit model
EPOCHS=50
BATCH_SIZE=64
model.fit(x_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE)
model.evaluate(x_test, y_test, verbose=0)

model.save(MODEL_TF_DIR)

11501568/11490434 [==============================] - 0s 0us/step
(60000, 28, 28) <class 'numpy.ndarray'>
Epoch 1/50
938/938 [==============================] - 17s 11ms/step - loss: 0.4217 - accuracy: 0.8746
Epoch 2/50
938/938 [==============================] - 11s 11ms/step - loss: 0.1453 - accuracy: 0.9572
Epoch 3/50
938/938 [==============================] - 10s 11ms/step - loss: 0.1070 - accuracy: 0.9683
Epoch 4/50
938/938 [==============================] - 9s 10ms/step - loss: 0.0868 - accuracy: 0.9740
Epoch 5/50
938/938 [==============================] - 7s 8ms/step - loss: 0.0738 - accuracy: 0.9771
Epoch 6/50
938/938 [==============================] - 7s 8ms/step - loss: 0.0655 - accuracy: 0.9799
Epoch 7/50
938/938 [==============================] - 8s 9ms/step - loss: 0.0576 - accuracy: 0.9819
Epoch 8/50
938/938 [==============================] - 9s 10ms/step - loss: 0.0522 - accuracy: 0.9841
Epoch 9/50
938/938 [==============================] - 7s 8ms/step - loss: 0.0492 - accu

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/ONNX/model_MNIST/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/ONNX/model_MNIST/assets


**Convert to TFLite and save**

# Convert model to ONNX

In [7]:
import tf2onnx
import onnxruntime as ort
import onnx

In [37]:
#@title From model after trained (not saved model) 
# load model
model = tf.keras.models.load_model(MODEL_TF_DIR)
model.summary()

# Use from_function for tf functions
onnx_model, _ = tf2onnx.convert.from_keras(model, opset=13)
onnx.save(onnx_model, MODEL_TF2ONNX_DIR)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 28, 20)            3920      
                                                                 
 flatten (Flatten)           (None, 560)               0         
                                                                 
 output (Dense)              (None, 10)                5610      
                                                                 
Total params: 9,530
Trainable params: 9,530
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#@title From saved model (model.pb) (OTHER)
# !python -m tf2onnx.convert --saved-model $MODEL_TF_DIR --output $MODEL_TF2ONNX_DIR --opset 13

# Test models

Tensorflow model vs ONNX model

In [46]:
model_tf = tf.keras.models.load_model(MODEL_TF_DIR)

sess = ort.InferenceSession(MODEL_TF2ONNX_DIR)
input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name

TEST_CASES = len(x_test)
expected = model.predict(x_test)

result = sess.run([label_name], {input_name:x_test})  # get the predict of ONNX model
count=0
for ort_res, tf_res in zip(expected , result ):
  try:
    np.testing.assert_allclose(ort_res, tf_res, rtol=1e-5, atol=1e-5)
  except:
    count+=1

print("Results have {count} not match in total {total} sample".format(count=count, total=len(x_test)))

Results have 1 not match in total 10000 sample


# scikit-learn to ONNX

In [32]:
!pip install skl2onnx 

     |████████████████████████████████| 273 kB 4.3 MB/s 
     |████████████████████████████████| 78 kB 6.4 MB/s 


Train a RF-Classifier model with IRIS dataset 

In [33]:
# Train a model.
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier


iris = load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y)
clr = RandomForestClassifier()
clr.fit(X_train, y_train)

RandomForestClassifier()

Convert model to ONNX

In [34]:
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

# convert model
initial_type = [('float_input', FloatTensorType([None, 4]))]
onx = convert_sklearn(clr, initial_types=initial_type)

# save model
with open("/content/rf_iris.onnx", "wb") as f:
    f.write(onx.SerializeToString())



In [49]:
# Compute the prediction with ONNX Runtime
import onnxruntime as rt
import numpy

sess = rt.InferenceSession("/content/rf_iris.onnx")
input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name
pred_onx = sess.run([label_name], {input_name: X_test.astype(numpy.float32)})


# pred of sklearn model
pred_skl = clr.predict(X_test)

count=0
for ort_res, tf_res in zip(pred_skl, pred_onx):
  try:
    np.testing.assert_allclose(ort_res, tf_res, rtol=1e-5, atol=1e-5)
  except:
    count+=1

print("Results have {count} not match in total {total} sample".format(count=count, total=len(X_test)))

Results have 1 not match in total 38 sample
